In [ ]:
#pip install cmake
#pip install dlib-19.23.0-cp39-cp39-win_amd64.whl
#pip install opencv-python
#pip install face_recognition

In [ ]:
#https://pypi.org/project/face-recognition/

In [4]:
import cv2
import face_recognition
import numpy as np

# Carregar as imagens de treinamento e rotulá-las
def load_images_and_labels(image_paths, labels):
    encodings = []
    for image_path, label in zip(image_paths, labels):
        image = face_recognition.load_image_file(image_path)
        encoding = face_recognition.face_encodings(image)[0]
        encodings.append((encoding, label))
    return encodings

image_paths = ["C:/Users/Magda/Downloads/eu.jpg", 
               "C:/Users/Magda/Downloads/selfie_mae.jpeg",
               "C:/Users/Magda/Downloads/fotos_turma/bruna.png",
               "C:/Users/Magda/Downloads/fotos_turma/eliane.png",
               "C:/Users/Magda/Downloads/fotos_turma/janaina.png"]

labels = ["Tainy", "Tania","Bruna","Eliane","Janaina"]

known_encodings = load_images_and_labels(image_paths, labels)

video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    rgb_frame = frame[:, :, ::-1]
    
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces([enc[0] for enc in known_encodings], face_encoding)
        name = "Desconhecido"

        face_distances = face_recognition.face_distance([enc[0] for enc in known_encodings], face_encoding)
        best_match_index = np.argmin(face_distances)

        if matches[best_match_index]:
            name = known_encodings[best_match_index][1]
        
        top, right, bottom, left = face_location
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1)
    
    cv2.imshow('Video', frame)

    # Pressione 'q' para sair do loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
